# Azizi lab
# Lingting Shi 1/6/2023
# Decipher analysis on GVHD project's all T cell data
# Raw counts

In [ ]:
#!pip install scdecipher

In [ ]:
import warnings; warnings.filterwarnings("ignore") # remove scanpy warnings for the tutorial.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scanpy as sc
import anndata

In [ ]:
import decipher as dc
dc.pl.activate_journal_quality()

In [ ]:
# Load the AnnData
adata =sc.read('/burg/iicd/users/ls3456/GVHD/Final_pipeline/Paper_ready_pipeline/anndata_file/combined_unfiltered_newdata.h5ad')

In [ ]:
adata.obs_names_make_unique()

In [ ]:
# Load in the anndata with doublet information
Tcell_sc = sc.read('/burg/iicd/users/ls3456/GVHD/Final_pipeline/Paper_ready_pipeline/anndata_file/tcell_ann_concat_Tcell_only_Migration.h5ad')
Tcell_sc

In [ ]:
Tcell = adata[Tcell_sc.obs.index]

In [ ]:
Tcell

In [ ]:
if list(Tcell.obs.index) == list(Tcell_sc.obs.index):
    print("The lists are identical")
else:
    print("The lists are not identical")

In [ ]:
Tcell.obs= Tcell_sc.obs

In [ ]:
Tcell.uns= Tcell_sc.uns

In [ ]:
Tcell.obsm= Tcell_sc.obsm

In [ ]:
Tcell.obs['leiden_48']

In [ ]:
Tcell.uns["color_palette"] = {'CD8': '#1f77b4',
 'CD4': '#ff7f0e',
 'Double_Pos': '#2ca02c'}

In [ ]:
color_palette = Tcell.uns["color_palette"]

In [ ]:
color_palette

In [ ]:
Tcell.uns["leiden_palette"] = {'0': '#023fa5',
 '1': '#7d87b9',
 '2': '#bec1d4','3': '#d6bcc0',
 '4': '#bb7784','5': '#8e063b',
 '6': '#4a6fe3','7': '#8595e1',
 '8': '#b5bbe3','9': '#e6afb9'}

In [ ]:
#Tcells_sc.uns["cd48_palette"] = {'CD8_class': '#1f77b4',
 #'CD4_class': '#ff7f0e'}

color_palette = Tcell.uns["color_palette"]
color_palette

In [ ]:
#dc.tl.DecipherConfig()

In [ ]:
# Tested with good seperation on epoches 15
decipher_config = dc.tl.DecipherConfig(n_epochs=15) # optional Decipher config

d1, val1 = dc.tl.decipher_train(
    Tcell, decipher_config, plot_kwargs={"color": "leiden_48", "palette": color_palette}, plot_every_k_epochs=5,
)

In [ ]:
plt.plot(val1)

In [ ]:
Tcell

In [ ]:
list(Tcell.obs['PatientID'].unique())

In [ ]:
dc.tl.decipher_rotate_space(
    Tcell,
    v1_col='leiden_48',
    v1_order=['CD8', 'CD4', 'Double_Pos'],
    v2_col='PatientID',
    v2_order=['R096','C155','C152','C159','C162','C068','C101','C157','ND001','C98','R085','C66','C86','C160','C179','ND003'],
)
dc.pl.decipher(Tcell, ['leiden_48'], palette=color_palette);

In [ ]:
dc.tl.cell_clusters(Tcell, leiden_resolution=0.05, n_neighbors=10, seed=0, rep_key='decipher_z')
dc.pl.decipher(Tcell, ["decipher_clusters"], legend_loc="on data");

In [ ]:
color_palette = Tcell.uns["leiden_palette"]

In [ ]:
Tcell.obs['leiden_0.5']

In [ ]:
color= ['leiden']

for meta in color:
    randomize=np.arange(len(Tcell.obs[meta]))
    np.random.shuffle(randomize)
    fig, ax = plt.subplots(figsize=(6,6),dpi=300)
    sns.set_style("whitegrid", {'axes.grid' : False})
    scatter=sns.scatterplot(x=Tcell.obsm["decipher_v"][:,0][randomize],y=Tcell.obsm["decipher_v"][:,1][randomize], hue=Tcell.obs[meta][randomize],s=2, palette=color_palette).set(title=meta)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    ax.set_xlabel('Decipher 1')
    ax.set_ylabel('Decipher 2')

In [ ]:
Tcell.var['mt'] = Tcell.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(Tcell, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
color=['GVHD_Grade_1',
            'LPorIE',
            'Tissue_Location',
             "leiden_0.5",
             'Transplant_Conditioning',
             'Donor',
             'Time_1stGVHD',
             "cellType_Terminal\nexhaustion_markers",
             'cellType_Cytolytics \neffector pathway_markers',
             'cc_aa_alignment_same_v_size',
             'clonal_expansion',
             'PatientID',
             'total_counts',
             'pct_counts_mt',
             'total_counts_mt',
             'cellType_G2/M_markers', 
             'cellType_G1/S_markers']

for meta in color:
    randomize=np.arange(len(Tcell.obs[meta]))
    np.random.shuffle(randomize)
    fig, ax = plt.subplots(figsize=(6,6),dpi=300)
    sns.set_style("whitegrid", {'axes.grid' : False})
    scatter=sns.scatterplot(x=Tcell.obsm["decipher_v"][:,0][randomize],y=Tcell.obsm["decipher_v"][:,1][randomize], hue=Tcell.obs[meta][randomize],s=4).set(title=meta)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    ax.set_xlabel('Decipher 1')
    ax.set_ylabel('Decipher 2')

In [ ]:
color=['cc_aa_alignment_same_v_size',
             'clonal_expansion',
             'PatientID',
             'total_counts',
             'pct_counts_mt',
             'total_counts_mt']

for meta in color:
    randomize=np.arange(len(Tcell.obs[meta]))
    np.random.shuffle(randomize)
    fig, ax = plt.subplots(figsize=(6,6),dpi=300)
    sns.set_style("whitegrid", {'axes.grid' : False})
    scatter=sns.scatterplot(x=Tcell.obsm["decipher_v"][:,0][randomize],y=Tcell.obsm["decipher_v"][:,1][randomize], hue=Tcell.obs[meta][randomize],s=4).set(title=meta)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    ax.set_xlabel('Decipher 1')
    ax.set_ylabel('Decipher 2')

In [ ]:
color=['GZMK','GZMB','TIGIT']

color=['CD3D','CD4','CD8A']

dc.pl.decipher(
    Tcell,
    color,
    ncols=3,
    vmax=lambda x: np.quantile(x, 0.99),c
    figsize=(6, 6),
    axis_type=None
);

In [ ]:
Tcell.write('/burg/iicd/users/ls3456/GVHD/Final_pipeline/Paper_ready_pipeline/anndata_file/Tcells_sc_dc.h5ad')

In [ ]:
Tcell